In [6]:
import numpy as np
import torch_geometric.nn
import torch_geometric.data as data
import networkx as nx
from torch_geometric.utils.convert import to_networkx

import torch
import torch.nn as nn
import scanpy as sc
import pandas as pd
import random
import argparse


from torch_geometric.nn import TransformerConv
import os

In [7]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def parse_args():
    parser = argparse.ArgumentParser(description='Run encoder')

    parser.add_argument('--epoches', type=int, default=2000,
                        help='number of epoches')
    parser.add_argument('--lambdac', type=float, default=0.01,
                        help='weight for the contrastive learning') 
    parser.add_argument('--lr1', type=float, default=1e-4,
                        help='lr for encoder')     
    parser.add_argument('--lr2', type=float, default=1e-3,
                        help='lr for decoder')
    parser.add_argument('--temp', type=float, default=0.07,
                        help='temperature for the contrastive learning') 
    parser.add_argument('--samplesize', type=int, default=100,
                        help='sample size for contrastive learning')
    parser.add_argument('--dim', type=float, default=32,
                        help='latent dimensions') 
    parser.add_argument('--savepath', type=str, default="heart_global/heart_umi_m_musegnn.h5ad",
                        help='save path') 
    
    
    return parser.parse_args()

In [8]:
import sys
import torch
import argparse
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#set_seed(0)

# # for specific encoder/decoder
# # tissue_list = { 
# #                "heart":[233, 676, 783, 947,266, 223, 233, 978, 928, 852, 839, 733]}
sys.argv = [
    'script_name',  # The first element is the script name, can be any name.
    '--epoches', '3',
    '--lambdac', '0.01',
    '--lr1', '1e-4',
    '--lr2', '1e-3',
    '--temp', '0.07',
    '--samplesize', '100',
    '--dim', '32',
    '--savepath', 'heart_umi_m_musegnn.h5ad'
]

# Now, you can call your function and parse the arguments
args = parse_args()

# Display the arguments to confirm that they are parsed correctly
print(args)

Namespace(epoches=3, lambdac=0.01, lr1=0.0001, lr2=0.001, temp=0.07, samplesize=100, dim=32.0, savepath='heart_umi_m_musegnn.h5ad')


In [9]:
tissue_list = {"heart":['rna', "spatial"]}

# construct graph batch
# based on simulation results
graph_list = []
cor_list = []
label_list = []
graph_networkx_list = []
count = 0
gene_length = 0

In [10]:
tissue_list

{'heart': ['rna', 'spatial']}

In [11]:
for tissue in tissue_list.keys():
    for i in tissue_list[tissue]:
        print(i)
        #pathway_count = "/projectnb/cs598/projects/singleCell/data/heart_spatial_expression.csv"
        #pathway_matrix = "/projectnb/cs598/projects/singleCell/data/heart_spatial_pvalue.csv"
        pathway_count = "/projectnb/cs598/projects/singleCell/data/heart_" + i + "_expression.csv"
        pathway_matrix = "/projectnb/cs598/projects/singleCell/data/heart_" + i + "_pvalue.csv"

        pd_adata_new =  pd.read_csv(pathway_count, index_col=0)
        correlation = pd.read_csv(pathway_matrix, index_col=0)
        cor_list.append(correlation)

        print(correlation.shape)
        print(pd_adata_new.shape)
        adata = sc.AnnData(pd_adata_new)
        gene_length = len(adata)

        adata_new = adata.copy()
        edges_new = np.array([np.nonzero(correlation.values)[0],np.nonzero(correlation.values)[1]])
        graph = data.Data(x=torch.FloatTensor(adata_new.X.copy()), edge_index=torch.FloatTensor(edges_new).long())
        
        vis = nx.from_pandas_adjacency(correlation)
        graph_networkx_list.append(vis)
        
        graph.gene_list = pd_adata_new.index
        graph.show_index = tissue +"__" + str(i)
        
        graph_list.append(graph)
        label_list.append(tissue)
        
        count +=1

rna
(1000, 1000)
(1000, 5000)
spatial
(1000, 1000)
(1000, 5000)


In [12]:
class GCNEncoder_Multiinput(torch.nn.Module):
    def __init__(self, out_channels, graph_list, label_list):
        super(GCNEncoder_Multiinput, self).__init__()
        self.activ = nn.Mish(inplace=True)
        
        conv_dict = {}
        conv_dict1 = {}
        for i in graph_list:
            conv_dict[i.show_index] = torch_geometric.nn.Sequential('x, edge_index', [(TransformerConv(i.x.shape[1], out_channels, heads=4),'x, edge_index-> x'),
                                                     (torch_geometric.nn.GraphNorm(out_channels*4), 'x -> x')])
        self.convl1 = nn.ModuleDict(conv_dict)
        self.convl2 = nn.ModuleDict(conv_dict1)
              
    def forward(self, x, edge_index, show_index):
        x = self.convl1[show_index](x, edge_index)
        x = self.activ(x)
        return x
    
class GCNEncoder_Commoninput(torch.nn.Module):
    def __init__(self, out_channels, graph_list, label_list):
        super(GCNEncoder_Commoninput, self).__init__()
        self.activ = nn.Mish(inplace=True)
        
        conv_dict_l2 = {}
        conv_dict_l3 = {}
        conv_dict_l4 = {}
        tissue_specific_list = list(set(label_list))
        
        for i in tissue_specific_list:
            conv_dict_l2[i] = torch_geometric.nn.Sequential('x, edge_index', [(TransformerConv(out_channels*4, out_channels, heads=2),'x, edge_index -> x'),
                                                     (torch_geometric.nn.GraphNorm(out_channels*2), 'x -> x')])
            conv_dict_l3[i] = TransformerConv(out_channels*2, out_channels)
            conv_dict_l4[i] = TransformerConv(out_channels*4, out_channels)
              
        self.convl2 = nn.ModuleDict(conv_dict_l2)
        self.convl3 = nn.ModuleDict(conv_dict_l3)
        self.convl4 = nn.ModuleDict(conv_dict_l4)
        
    def forward(self, x, edge_index, show_index):
        x_inp = x
        x = self.convl2[show_index.split('__')[0]](x, edge_index)
        x = self.activ(x)
        x = self.convl3[show_index.split('__')[0]](x, edge_index)
        return x + self.convl4[show_index.split('__')[0]](x_inp, edge_index)

class MLP_edge_Decoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, graph_list):
        super(MLP_edge_Decoder, self).__init__()
        
        dec_dict = {}
        
        self.activ = nn.Mish(inplace=True)
        for i in graph_list:
            dec_dict[i.show_index] = torch.nn.Sequential(
                                              nn.Linear(in_channels,  out_channels)
                                             , self.activ,
                                              nn.Linear(in_channels,  out_channels)
                                             , self.activ,
                                              nn.Linear(in_channels,  out_channels)
                                             )
        self.MLP = nn.ModuleDict(dec_dict)
        
    def forward(self, x, show_index):
        x = self.MLP[show_index](x)
        return x



In [13]:
graph_list

[Data(x=[1000, 5000], edge_index=[2, 311364], gene_list=Index(['ENSG00000187608', 'ENSG00000171621', 'ENSG00000162444',
        'ENSG00000175206', 'ENSG00000120937', 'ENSG00000189337',
        'ENSG00000231606', 'ENSG00000188257', 'ENSG00000173372',
        'ENSG00000159189',
        ...
        'ENSG00000154734', 'ENSG00000156273', 'ENSG00000156299',
        'ENSG00000159216', 'ENSG00000157554', 'ENSG00000198888',
        'ENSG00000198899', 'ENSG00000198840', 'ENSG00000198886',
        'ENSG00000198727'],
       dtype='object', length=1000), show_index='heart__rna'),
 Data(x=[1000, 5000], edge_index=[2, 206998], gene_list=Index(['ENSG00000188290', 'ENSG00000187608', 'ENSG00000162576',
        'ENSG00000179403', 'ENSG00000197785', 'ENSG00000189409',
        'ENSG00000235169', 'ENSG00000116285', 'ENSG00000074800',
        'ENSG00000171819',
        ...
        'ENSG00000159200', 'ENSG00000185437', 'ENSG00000157601',
        'ENSG00000160214', 'ENSG00000160255', 'ENSG00000182871',
      

In [14]:
gene_encoder_is = GCNEncoder_Multiinput(int(args.dim), graph_list, label_list).to(device)
gene_encoder_com =  GCNEncoder_Commoninput(int(args.dim), graph_list, label_list).to(device)

gene_decoder = MLP_edge_Decoder(gene_length, gene_length ,graph_list).to(device)

optimizer_enc_is = torch.optim.Adam(gene_encoder_is.parameters(), lr=args.lr1)
optimizer_enc_com = torch.optim.Adam(gene_encoder_com.parameters(), lr=args.lr1)

optimizer_dec2 = torch.optim.Adam(gene_decoder.parameters(), lr=args.lr2)

import numpy as np
import networkx as nx
from concurrent.futures import ProcessPoolExecutor,as_completed, ThreadPoolExecutor
from multiprocessing import cpu_count

import process_pair

def compute_gene_sets(graph_list, graph_networkx_list, num_threads=cpu_count()):
    common_gene_set = {}
    common_gene_overlap = {}
    diff_gene_set = {}
    diff_gene_neighbor_set = {}

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [
            executor.submit(process_pair.process_pair, i, j, graph_list, graph_networkx_list)
            for i in range(len(graph_list))
            for j in range(len(graph_list))
            if i != j
        ]

        for future in as_completed(futures):
            i, j, common_set, common_overlap, diff_set, diff_neighbor_set = future.result()
            key = graph_list[i].show_index + graph_list[j].show_index
            common_gene_set[key] = common_set
            common_gene_overlap[key] = common_overlap
            diff_gene_set[key] = diff_set
            diff_gene_neighbor_set[key] = diff_neighbor_set

    return common_gene_set, common_gene_overlap, diff_gene_set, diff_gene_neighbor_set
common_gene_set, common_gene_overlap, diff_gene_set, diff_gene_neighbor_set = compute_gene_sets(graph_list, graph_networkx_list)

print("finish preprocessing")
print("start training")

loss_f = nn.BCEWithLogitsLoss()
loss_m = nn.CrossEntropyLoss()
from pytorch_metric_learning import losses
loss_func = losses.SelfSupervisedLoss(losses.NTXentLoss(temperature = args.temp))

lambda_infonce = args.lambdac
def penalize_data(z, graph_list,j):
    loss = torch.tensor(0.).to(device)
    graph_new = graph_list[j]

    x = graph_new.x.to(device)
    train_pos_edge_index = graph_new.edge_index.to(device)
    
    x = gene_encoder_is(x, train_pos_edge_index, graph_new.show_index)
    z_new = gene_encoder_com(x, train_pos_edge_index, graph_new.show_index)

    [index_i, index_j] = common_gene_set[graph.show_index + graph_new.show_index]
    if (len(index_i) ==0) or (len(index_j) == 0):
        return loss
    
    z_cor = z[index_i]
    z_new_cor = z_new[index_j]
    
    weight = torch.FloatTensor(common_gene_overlap[graph.show_index + graph_new.show_index]).to(device)
    cos_sim = torch.cosine_similarity(z_cor, z_new_cor, axis = 1)*weight

    loss += -cos_sim.mean()
    
    [index_i, index_j] = diff_gene_set[graph.show_index + graph_new.show_index]

    if (len(index_i) ==0) or (len(index_j) == 0):
        return loss
    
    opt_index = np.random.choice([i for i in range(len(index_i))], min(100, len(index_i)))
    
    z_diff = z[index_i[opt_index]]
    z_new_diff = z_new[index_j[opt_index]]
    
    [index_i, index_j] = diff_gene_neighbor_set[graph.show_index + graph_new.show_index]
    
    z_diff_true =  z[index_i[opt_index]]
    z_new_diff_true = z_new[index_j[opt_index]]
    
    loss += lambda_infonce * loss_func(torch.cat((z_diff,z_new_diff)), torch.cat((z_diff_true,z_new_diff_true)))
    return loss



finish preprocessing
start training


In [15]:
# Contrastive learning
gene_encoder_is.train()
gene_encoder_com.train()
graph_index_list = [item for item in range(0, len(graph_list))]
edge_adj_list = [torch.FloatTensor(cor_list[i].values).to(device) for i in graph_index_list]

for epoch in range(args.epoches):
    loss = 0.
    for i in range(0,len(graph_index_list)):
        
        optimizer_enc_is.zero_grad(set_to_none=True)
        optimizer_enc_com.zero_grad(set_to_none=True)
        optimizer_dec2.zero_grad(set_to_none=True)

        graph = graph_list[i]

        x = graph.x.to(device)
        train_pos_edge_index = graph.edge_index.to(device)
        edge_adj = edge_adj_list[i]

        x = gene_encoder_is(x, train_pos_edge_index,  graph.show_index)
        z = gene_encoder_com(x, train_pos_edge_index, graph.show_index)
        
        adj = torch.matmul(z, z.t())
        edge_reconstruct = gene_decoder(adj, graph.show_index)
        
        loss = loss_f(edge_reconstruct.flatten(), edge_adj.flatten()) 

        if epoch % 200 ==0:
            print(loss.item())

        graph_index_list_copy = graph_index_list.copy()
        graph_index_list_copy.remove(i)

        j = random.sample(graph_index_list_copy, 1)
        loss += penalize_data(z, graph_list,j[0]) 

        del graph
        loss.backward()
        del loss
        
        optimizer_enc_is.step()
        optimizer_enc_com.step()
        optimizer_dec2.step()
        
    print("epoch finish")

torch.cuda.empty_cache()

emb_list = []
gene_list = []
tissue_list = []

# inference step
gene_encoder_is.eval()
gene_encoder_com.eval()
with torch.no_grad():
    for i in range(0,len(graph_list)):
        graph = graph_list[i]
        x = graph.x.to(device)
        train_pos_edge_index = graph.edge_index.long().to(device)
        
        x = gene_encoder_is(x, train_pos_edge_index,  graph.show_index)
        z = gene_encoder_com(x, train_pos_edge_index, graph.show_index)
        
        emb_list.append(z.cpu().numpy())
        gene_list.append(graph.gene_list)
        tissue_list.append([graph.show_index for j in range(len(x))])

adata = sc.AnnData(np.concatenate(emb_list))

adata.obs['gene'] = np.concatenate(gene_list)
adata.obs['tissue'] = np.concatenate(tissue_list)

sc.pp.neighbors(adata, use_rep='X')
sc.tl.umap(adata)

tissue_list1 = []
for i in list(set(adata.obs['tissue'])):
    tissue_list1.append(list(adata[adata.obs['tissue'] == i].obs['gene']))

common_gene_list = set(tissue_list1[0]).intersection(*tissue_list1[1:])

adata.obs['displaygene']= [True if i in common_gene_list else False for i in adata.obs['gene']]
adata.obs['displaygene']  = adata.obs['displaygene'].astype('category')

sc.tl.leiden(adata)

adata.obs['tissue_new'] = [i.split("__")[0] for i in adata.obs['tissue']]

adata.write_h5ad(args.savepath)

0.7049847841262817
0.7128649950027466
epoch finish
epoch finish
epoch finish


/rprojectnb/apoe-signatures/aleshchk/.conda/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/scratch/3289030.1.csgpu/ipykernel_113821/1320484080.py:87: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(adata)


In [21]:
#sc.pp.neighbors(adata, use_rep='X')
#sc.tl.umap(adata)
adata.obs['tissue']
tissue_list = {"heart":['rna', "spatial"]}

In [22]:
import scanpy as sc
import numpy as np
import pandas as pd
import networkx as nx

import scib

from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score


# construct graph batch based on simulation results
graph_list = []
cor_list = {}
label_list = [] 
count = 0
for tissue in tissue_list.keys():
    for i in tissue_list[tissue]:
        print(i)
        pathway_count = "/projectnb/cs598/projects/singleCell/data/heart_" + i + "_expression.csv"
        pathway_matrix = "/projectnb/cs598/projects/singleCell/data/heart_" + i + "_pvalue.csv"
        correlation = pd.read_csv(pathway_matrix, sep=",", index_col=0)
        cor_list[tissue +"__" + str(i)] = correlation

        print(correlation.shape)

        label_list.append(tissue +"__" + str(i))
        
        count +=1

# sigmoid function
def sigmoid(x):
    return 1/(1 + np.exp(-x))

# metric 1: Calculate gene ASW
def calculate_common_asw(adata):
    tissue_list = []
    for i in list(set(adata.obs['tissue'])):
        tissue_list.append(list(adata[adata.obs['tissue'] == i].obs['gene']))
        
    common_gene_list = set(tissue_list[0]).intersection(*tissue_list[1:])
    adata_new = adata[[True if i in common_gene_list else False for i in adata.obs['gene']]]
    adata_new.obsm['X_emb'] = adata_new.X
    
    result = scib.metrics.silhouette_batch(adata_new, batch_key='tissue', group_key='leiden', embed='X_emb')
    
    return result    


# metric 2: Calculate normalized AUC
def calculate_AUC(adata, cor_list):
    tissue_list = list(set(cor_list.keys()))
    
    result = 0
    for i in tissue_list:
        adata_new = adata[adata.obs['tissue'] == i]

        normed_matrix = normalize(adata_new.X, axis=1)
        rec_matrix = sigmoid(normed_matrix@normed_matrix.T).flatten()
        cor_matrix = cor_list[i].values.flatten()
        result += roc_auc_score(cor_matrix, rec_matrix)
    
    result = result/len(tissue_list)
    return result    

# metric 3: Calculate gene LISI
def calculate_iLISI(adata):
    tissue_list = []
    for i in list(set(adata.obs['tissue'])):
        tissue_list.append(list(adata[adata.obs['tissue'] == i].obs['gene']))
        
    common_gene_list = set(tissue_list[0]).intersection(*tissue_list[1:])
    adata_new = adata[[True if i in common_gene_list else False for i in adata.obs['gene']]]
    adata_new.obsm['X_emb'] = adata_new.X
    
    result = scib.metrics.ilisi_graph(adata_new, batch_key="tissue", type_="embed")
    
    return result   

# metric 4: Calculate gene GC
def calculate_graph_connectivity(adata):
    tissue_list = []
    for i in list(set(adata.obs['tissue'])):
        tissue_list.append(list(adata[adata.obs['tissue'] == i].obs['gene']))
        
    common_gene_list = set(tissue_list[0]).intersection(*tissue_list[1:])
    adata_new = adata[[True if i in common_gene_list else False for i in adata.obs['gene']]]

    adata_new.obsm['X_emb'] = adata_new.X
    
    result = scib.metrics.graph_connectivity(adata_new,'leiden')
    
    return result    

# metric 5: Calculate commom gene propertion
def calculate_common_gene_propertion(adata):
    full_score = 0
    for i in list(set(adata.obs['leiden'])):
        adata_new = adata[adata.obs['leiden'] == i]
        
        gene_list = set(adata_new.obs['gene'])
        
        prop = 1 - len(gene_list)/len(adata_new.obs['gene'])
        print(prop)
        
        full_score += len(adata_new)/len(adata) * prop
        
    return full_score

# Helper function: Jaccard score for genes in different graphs.
def calculate_overlap(G1,G2,g1,g2):
    G1_neg = list(G1.neighbors(g1))
    G2_neg = list(G2.neighbors(g2))
    
    overlap_score = len(set(G1_neg).intersection(set(G2_neg)))/len(set(G1_neg).union(set(G2_neg)))
    
    return overlap_score

# metric 6: Calculate neighbor genes' overlap
def calculate_common_neighbor_ovarlap(adata, cor_list):
    output_value = 0
    for i in list(set(adata.obs['leiden'])):
        adata_new = adata[adata.obs['leiden'] == i]
        
        tissue_list = list(adata_new.obs['tissue'])
        gene_list = list(adata_new.obs['gene'])
        
        overlap_value = 0
        dim_value = 0
        for num1,item1 in enumerate(gene_list):
            for num2,item2 in enumerate(gene_list):
                t1 = tissue_list[num1]
                t2 = tissue_list[num2]
                if t1 != t2:
                    g1 = graph_list[t1]
                    g2 = graph_list[t2]
                    temp_overlap = calculate_overlap(g1,g2,item1,item2)
                    overlap_value += temp_overlap
                    dim_value += 1.0
                
        print("finish one cluster")
        
        if dim_value == 0:
            overlap_value = 0
        else:
            overlap_value = overlap_value/dim_value
        output_value += overlap_value*len(adata_new)/len(adata)
        
    return output_value

# Integrated function for metric calculation
def calculate_metric(adata, cor_list):
    asw = calculate_common_asw(adata)
    AUC = calculate_AUC(adata, cor_list)
    #ilisi = calculate_iLISI(adata)
    gc = calculate_graph_connectivity(adata)
    
    
    ratio = calculate_common_gene_propertion(adata)
    
    ovl = calculate_common_neighbor_ovarlap(adata, cor_list)
    
    #df = pd.DataFrame(np.array([asw,AUC,ilisi,gc,ratio, ovl]))
    df = pd.DataFrame(np.array([asw, AUC,gc,ratio, ovl]))
    #df.index = ['ASW', 'AUC', 'iLISI', 'GC', 'Common ratio', 'share overlap']
    df.index = ['ASW', 'AUC', 'GC', 'Common ratio', 'share overlap']
    print(df)
    return df

# run the benchmark process
graph_list = {}
for i in cor_list.keys():
    graph_list[i] = nx.from_pandas_adjacency(cor_list[i])

# run the benchmark
seed = 0
adata = sc.read_h5ad("heart_umi_m_musegnn.h5ad")
calculate_metric(adata, cor_list)

rna
(1000, 1000)
spatial
(1000, 1000)


/scratch/3289030.1.csgpu/ipykernel_113821/389374859.py:43: ImplicitModificationWarning: Setting element `.obsm['X_emb']` of view, initializing view as actual.
  adata_new.obsm['X_emb'] = adata_new.X
/scratch/3289030.1.csgpu/ipykernel_113821/389374859.py:153: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  asw = calculate_common_asw(adata)


mean silhouette per group:        silhouette_score
group                  
0              0.967297
1              0.818709
11             0.626797
2              0.866966
3              0.969360
4              0.887299
5              0.928222
6              0.899034
7              0.875465
8              0.909301


/scratch/3289030.1.csgpu/ipykernel_113821/389374859.py:89: ImplicitModificationWarning: Setting element `.obsm['X_emb']` of view, initializing view as actual.
  adata_new.obsm['X_emb'] = adata_new.X
/rprojectnb/apoe-signatures/aleshchk/.conda/envs/myenv/lib/python3.12/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
/rprojectnb/apoe-signatures/aleshchk/.conda/envs/myenv/lib/python3.12/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
/rprojectnb/apoe-signatures/aleshchk/.conda/envs/myenv/lib/python3.12/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Serie

0.13564668769716093
0.0
0.06976744186046513
0.0803571428571429
0.04471544715447151
0.0625
0.04878048780487809
0.015625
0.04489795918367345
0.04166666666666663
0.0
0.16666666666666663
finish one cluster
finish one cluster
finish one cluster
finish one cluster
finish one cluster
finish one cluster
finish one cluster
finish one cluster
finish one cluster
finish one cluster
finish one cluster
finish one cluster
                      0
ASW            0.874845
AUC            0.556394
GC             0.919824
Common ratio   0.064000
share overlap  0.058530


,0
ASW,0.874845
AUC,0.556394
GC,0.919824
Common ratio,0.064000
share overlap,0.058530
